In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import json
sns.set_context("notebook")

In [7]:
def load_sweep(path: str) -> pd.DataFrame:
    with open(path) as f:
        return pd.DataFrame.from_dict(
            dict(**line, **result)
            for line in map(json.loads, f)
            if "results" in line
            for result in line["results"]
        ).drop(columns=["results"])

(pd.concat([load_sweep(f) for f in ["data/sweep_v2.jsonl", "data/sweep_v2c.jsonl"]]).reset_index()
    .groupby(["dataset", "shots", "model_scale", "sparsity", "k", "local_ratio", "sparse_generation_only"], dropna=False, sort=False)[["match", "rougeL"]].mean()
    .reset_index()
    .to_json("summary_sparse_generation.json")
)


In [8]:
df = (pd.read_json("summary_sparse_generation.json")
    .pipe(lambda d: d.assign(score=np.where(d.match.isna(), d.rougeL, d.match.astype("float"))))
    .drop(columns=["shots", "k", "local_ratio", "match", "rougeL"])
)
print(len(df))

63


In [9]:
df.pivot(index=["dataset", "model_scale", "sparsity"], columns="sparse_generation_only", values="score")

sparse_generation_only                          False     True 
dataset       model_scale sparsity                             
cnn_dailymail 1.4b        none               0.181557       NaN
                          sparsev_after      0.199421  0.186093
                          sparsev_after_avg  0.178949  0.183197
                          sparsev_before     0.203826  0.194425
              1b          none               0.220002       NaN
                          sparsev_after      0.210284  0.218367
                          sparsev_after_avg  0.215882  0.200997
                          sparsev_before     0.213776  0.208244
              2.8b        none               0.219592       NaN
                          sparsev_after      0.202309  0.206064
                          sparsev_after_avg  0.213470  0.212581
                          sparsev_before     0.217122  0.219663
squad         1.4b        none               0.375000       NaN
                          sparsev_after      0.260000  0.375000
                          sparsev_after_avg  0.362500  0.375000
                          sparsev_before     0.385000  0.382500
              1b          none               0.375000       NaN
                          sparsev_after      0.207500  0.367500
                          sparsev_after_avg  0.265000  0.277500
                          sparsev_before     0.300000  0.340000
              2.8b        none               0.495000       NaN
                          sparsev_after      0.365000  0.495000
                          sparsev_after_avg  0.465000  0.492500
                          sparsev_before     0.462500  0.502500
triviaqa      1.4b        none               0.412500       NaN
                          sparsev_after      0.392500  0.432500
                          sparsev_after_avg  0.415000  0.417500
                          sparsev_before     0.370000  0.415000
              1b          none               0.322500       NaN
                          sparsev_after      0.342500  0.330000
                          sparsev_after_avg  0.320000  0.255000
                          sparsev_before     0.325000  0.282500
              2.8b        none               0.482500       NaN
                          sparsev_after      0.435000  0.495000
                          sparsev_after_avg  0.467500  0.480000
                          sparsev_before     0.445000  0.492500